<a href="https://colab.research.google.com/github/jeppa1/projeto-analise-territorial-rn/blob/main/enriquecer_mapa_rn_com_territorios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalação e importação das bibliotecas necessárias
!pip install geopandas
import pandas as pd
import geopandas as gpd
import os

In [4]:
# --- Configuração dos Caminhos ---
# Esta estrutura assume que o notebook está na pasta raiz do projeto
# e os dados estão na subpasta 'data/'

# Arquivos de entrada
caminho_geojson_original = "data/input/geojs-24-mun.json"
caminho_master_csv = "data/input/municipios_rn_territorio - territorios_master.csv"

# Pasta de saída
pasta_saida = "data/output/"
if not os.path.exists(pasta_saida):
    os.makedirs(pasta_saida)

# --- Carregamento dos Arquivos ---
try:
    print("Carregando arquivo GeoJSON original...")
    gdf_mapa = gpd.read_file(caminho_geojson_original)

    print("Carregando arquivo mestre de territórios...")
    df_master = pd.read_csv(caminho_master_csv, dtype={'ibge_json': str})
    print("--> Arquivos carregados com sucesso!")
except FileNotFoundError:
    print("ERRO: Arquivo não encontrado. Verifique a estrutura de pastas e os nomes dos arquivos.")
    raise

Carregando arquivo GeoJSON original...
Carregando arquivo mestre de territórios...
--> Arquivos carregados com sucesso!


In [5]:
# Junção dos dados
gdf_atualizado = gdf_mapa.merge(
    df_master,
    left_on='id',
    right_on='ibge_json',
    how='left'
)

# Limpeza e reorganização do GeoDataFrame final
gdf_final = gdf_atualizado[['nome_atualizado', 'territorio', 'ibge_json', 'geometry']].copy()
gdf_final.rename(columns={
    'nome_atualizado': 'municipio',
    'ibge_json': 'cod_ibge'
}, inplace=True)

print("\nPrévia dos dados do novo mapa enriquecido:")
print(gdf_final.head())


Prévia dos dados do novo mapa enriquecido:
        municipio                             territorio cod_ibge  \
0           Acari                                 Seridó  2400109   
1            Assu                           Assu/Mossoró  2400208   
2  Afonso Bezerra  Sertão Central Cabugi e Litoral Norte  2400307   
3       Água Nova                             Alto Oeste  2400406   
4      Alexandria                             Alto Oeste  2400505   

                                            geometry  
0  POLYGON ((-36.67528 -6.26957, -36.67217 -6.274...  
1  POLYGON ((-37.01502 -5.87045, -37.03524 -5.890...  
2  POLYGON ((-36.43954 -5.36949, -36.45491 -5.486...  
3  POLYGON ((-38.3087 -6.177, -38.28819 -6.18751,...  
4  POLYGON ((-37.90206 -6.29738, -37.90048 -6.317...  


In [6]:
# Otimização do tamanho do arquivo
gdf_otimizado = gdf_final.copy()
gdf_otimizado['geometry'] = gdf_final.geometry.simplify(tolerance=0.001)

# Salvando o novo arquivo GeoJSON
caminho_novo_geojson = os.path.join(pasta_saida, "RN_MUNICIPIOS_TERRITORIOS_OTIMIZADO.json")
gdf_otimizado.to_file(caminho_novo_geojson, driver='GeoJSON')

print("\n--- SUCESSO! ---")
print(f"Seu novo arquivo de mapa foi salvo em:")
print(caminho_novo_geojson)


--- SUCESSO! ---
Seu novo arquivo de mapa foi salvo em:
data/output/RN_MUNICIPIOS_TERRITORIOS_OTIMIZADO.json
